In [2]:
#utf-8 DES block cipher

import nbimporter,os
from DES_singleblock_encrypt import des_encrypt
from DES_singleblock_decrypt import des_decrypt

class DES():
    #initialization
    def __init__(self,mode):
        self.mode = mode
        self.enc_list = list()
        self.dec_list = list()
    
    #XOR operation 16 hex digits 
    def XOR(self,hex1,hex2):
        xor_result = str()
        bit1 = bin(int(hex1,16))[2:].zfill(64)
        bit2 = bin(int(hex2,16))[2:].zfill(64)
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r)
        
        xor_result_to_hex = hex(int(xor_result,2))[2:].zfill(16)
        return xor_result_to_hex
    
    def encrypt(self,plain_txt,k1,k2,k3,iv = None):
        '''block size = 8 bytes'''
        utf8_list = list()
        padding_str = str()
        grou_list = list()
        
        '''transform plaintext into hex string using utf-8'''
        for i in plain_txt:
            if ord(i) < 128:
            #ascii/codepoint 127 :delet
                utf8_list.append(hex(ord(i)).replace('0x','').zfill(2))
            else:
                #utf8
                utf8_cha1 = str(i.encode('utf8')).strip('b')
                utf8_cha2 = utf8_cha1.replace('\\x','')
                utf8_cha3 = utf8_cha2.strip('\'')
                utf8_list.append(utf8_cha3)
        utf8_str = ''.join(utf8_list)
        
        '''PKCS5 padding to reach multiple of 16 hex digits'''
        padding_str = utf8_str
        if len(utf8_str) % 16 != 0:
            remainder = len(utf8_str) % 16
            padding_num = (16-remainder) // 2
            for i in range(padding_num):
                padding_str += hex(padding_num)[2:].zfill(2)
    
        '''16 hex digits a group'''
        quotient = int(len(padding_str)/16)
        for j in range(quotient):
            item = padding_str[16*j]
            for k in range(15):
                item += padding_str[16*j + (k+1)]
            grou_list.append(item)

        '''mode operation'''
        if self.mode == 'ecb':
            for plain_16hex in grou_list:
                enc1 = des_encrypt(plain_16hex,k1)
                dec2 = des_decrypt(enc1.generate(),k2)
                enc3 = des_encrypt(dec2.generate(),k3)
                self.enc_list.append(enc3.generate())
            return ''.join(self.enc_list)
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv       
            for plain_16hex in grou_list:
                enc1 = des_encrypt(self.XOR(xor_ob,plain_16hex),k1)
                dec2 = des_decrypt(enc1.generate(),k2)
                enc3 = des_encrypt(dec2.generate(),k3)
                xor_ob = enc3.generate()
                self.enc_list.append(xor_ob)
            return ''.join(self.enc_list)
        
    def decrypt(self,cipher_txt,k1,k2,k3,iv = None):
        '''block size = 8 bytes'''
        grou_list = list()
        code_point_list = list()
        bin_str = str()
        
        '''16 hex digits a group'''
        quotient = int(len(cipher_txt)/16)
        for j in range(quotient):
            item = cipher_txt[16*j]
            for k in range(15):
                item += cipher_txt[16*j + (k+1)]
            grou_list.append(item)
 
        '''mode operation'''
        if self.mode == 'ecb':
            for cipher_16hex in grou_list:
                dec1 = des_decrypt(cipher_16hex,k3)
                enc2 = des_encrypt(dec1.generate(),k2)
                dec3 = des_decrypt(enc2.generate(),k1)
                self.dec_list.append(dec3.generate())
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv
            for cipher_16hex in grou_list: 
                dec1 = des_decrypt(cipher_16hex,k3)
                enc2 = des_encrypt(dec1.generate(),k2)
                dec3 = des_decrypt(enc2.generate(),k1)
    
                #XOR operation
                xor_result = self.XOR(xor_ob,dec3.generate())
                self.dec_list.append(xor_result)
                xor_ob = cipher_16hex
        
        '''2 hex digits a group in list'''
        dec_str = ''.join(self.dec_list)
        quotient = int(len(dec_str)/2)
        for i in range(quotient):
            item = dec_str[2*i] + dec_str[2*i+1]
            code_point_list.append(int(item,16))
        
        '''eliminate padding digits'''
        padding = True
        index,num_pad = len(code_point_list)-1,1
    
        while padding:
            if code_point_list[index] == code_point_list[index-1]:
                index = index-1
                num_pad = num_pad+1
            else:
                if num_pad == code_point_list[index]:
                    for i in range(num_pad):
                        code_point_list.pop()        
                padding = False
        
        '''utf8 decoding'''
        import bitstring as bit
        for j in code_point_list:
            #append 1 byte at a time
            bin_str = bin_str + bin(j)[2:].zfill(8)
    
        '''turn string type to bytes data type'''
        decode_to_bin = bit.BitArray('0b'+ bin_str)  
        decode_result = decode_to_bin.tobytes().decode('utf8')
        return decode_result
        

#-----------testing-----------------
plaintext = 'Test測試Ã cryptography'
k1 = '123456789ABCDEF0'
k2 = '23456789ABCDEF01'
k3 = '3456789ABCDEF012'
iv = '0123456789ABCDEF'

a = DES(mode='ecb')
a_ecb_cipher = a.encrypt(plaintext,k1,k2,k3)
a_ecb_decipher = a.decrypt(a_ecb_cipher,k1,k2,k3)

b = DES(mode='cbc')
b_cbc_cipher = b.encrypt(plaintext,k1,k2,k3,iv)
b_cbc_decipher = b.decrypt(b_cbc_cipher,k1,k2,k3,iv)

print({a_ecb_cipher:a_ecb_decipher})
print({b_cbc_cipher:b_cbc_decipher})

{'159B361C728B3DAD1E083F9826590A81753F42B2651A4787B323C85D6DEE632E': 'Test測試Ã cryptography'}
{'25AE9A7EDC7737E5B853B30A02C94A8E0D72BF791ED046BB308685F5AE9E1757': 'Test測試Ã cryptography'}
